In [ ]:
import pandas as pd 
import numpy as np 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup as bs
import requests
import os.path

### Competitors

In [ ]:
links = {
    "Genetec": "https://investingmalaysia.com/category/information-technology-industry/automation-solution/",
    "Unisem": "https://investingmalaysia.com/category/manufacturing-industry/semiconductor/"
}
companies_competitors = {
    "Genetec": [],
    "Unisem": []
}

In [ ]:
def competitors_scraper(links: dict) -> None:
    companies_competitors = {link: [] for link in links}
    for company, link in links.items():
        options = Options()
        options.add_argument("--headless")
        driver = webdriver.Chrome(options=options)
        driver.get(link)
        # res = requests.get(link)
        soup = bs(driver.page_source, "lxml")
        competitors = [competitor.text.split(" ")[0] for competitor in soup.select("h3.fontnormal.mb10.mt0.lineheight25 > a")]
        companies_competitors[company] = competitors
    return companies_competitors 

In [ ]:
companies_competitors = competitors_scraper(links)
companies_competitors

### Financial Statement

In [ ]:
link = "https://www.tradingview.com/symbols/MYX-sybl/statement/"
statements = ['financials-income-statement', 'financials-balance-sheet', "financials-cash-flow", "financials-statistics-and-ratios"]

In [ ]:
def get_driver(headless=False):
    options = Options()
    if headless == True:
        options.add_argument("--headless")
    return webdriver.Chrome(options=options)

### Elements Needed
title: .titleWrap-C9MdAMrq .titleText-C9MdAMrq <br/>
years: .values-OWKkVLyj.values-AtxjAQkN  .value-OxVAcLqi <br/>
values: .values-C9MdAMrq.values-AtxjAQkN  .value-OxVAcLqi <br/>

In [ ]:
def find(obj, selector):
    return WebDriverWait(obj, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
def finds(obj, selector):
    return WebDriverWait(obj, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, selector)))
def get_texts(objs):
    return [item.text for item in objs]

In [ ]:
## This code will take around 30-40 mins to complete running, depends on the wifi connection

dfs = {
    "Genetec": {
        "df_inc": None,
        "df_bal": None,
        "df_cash": None,
        "df_ratio": None,
    },
    "Unisem": {
        "df_inc": None,
        "df_bal": None,
        "df_cash": None,
        "df_ratio": None,
    }
}
stamement_name_converter = {
    "financials-income-statement": "df_inc",
    "financials-balance-sheet": "df_bal",
    "financials-cash-flow": "df_cash",
    "financials-statistics-and-ratios": "df_ratio"
}
## // Function 
headless = True

# for company, competitors in companies_competitors.items():
company = "Unisem"
competitors = list(companies_competitors.values())[-1]
for company, competitors in companies_competitors.items():
    for competitor in competitors:
        for statement in statements:
            driver = get_driver(headless=headless)
            driver.get(link.replace("sybl", competitor).replace("statement", statement))
            button = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "button[role='tab']")))[0]
            button.click()  # choose annual data
            sleep(.1)
            table = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.container-Tv7LSjUz"))) # wait table of statement to load

            def find_toggles(container): # find the toggles to be expanded
                return driver.execute_script("return document.querySelectorAll(arguments[0]);", ".arrow-C9MdAMrq")
            toggles = find_toggles(table)
            i = 0
            while True:   # click all toggles to scrape all hidden data
                if i == len(toggles):
                    break
                toggles = driver.execute_script("return document.querySelectorAll(arguments[0]);", ".arrow-C9MdAMrq")
                driver.execute_script("arguments[0].click()", toggles[i])
                i += 1

            try:
                tables = driver.execute_script("return document.querySelector(arguments[0]);", "div.container-Tv7LSjUz")
                titles = get_texts(finds(table, ".titleWrap-C9MdAMrq .titleText-C9MdAMrq")) # crawl title of each row

                years = get_texts(finds(table, ".values-OWKkVLyj.values-AtxjAQkN  .value-OxVAcLqi")) # crawl years available

                data_containers = finds(table, ".values-C9MdAMrq.values-AtxjAQkN") # scrape each row's data container
                frames= []
                for data_container in data_containers:
                    datas = [data.replace("\u202a", "").replace("\u202c", "") for data in get_texts(finds(data_container, ".value-OxVAcLqi"))]
                    frames.append(datas)
                years = years[-1 * len(frames[0]): ]

                if dfs[company][stamement_name_converter[statement]] is None:
                    dfs[company][stamement_name_converter[statement]] = pd.DataFrame(frames, columns=years, index= [(competitor, title) for title in titles])
                    df = dfs[company][stamement_name_converter[statement]]
                    df.index = pd.MultiIndex.from_tuples(df.index, names=['Company','Indicator'])
                else:
                    df = dfs[company][stamement_name_converter[statement]]
                    df2 = pd.DataFrame(frames, columns=years, index= [(competitor, title) for title in titles])
                    df.index = pd.MultiIndex.from_tuples(df.index, names=['Company','Indicator'])
                    df = pd.concat([df, df2])
                    dfs[company][stamement_name_converter[statement]] = df
            except:
                print(f"{company} -> {competitor} -> {statement}")
                break

In [ ]:
dfs["Genetec"]['df_bal'].index.get_level_values(1)

In [ ]:
genetec = dfs['Genetec'].copy()
unisem = dfs['Unisem'].copy()
companies = [genetec, unisem]

In [ ]:
dfs

In [ ]:
companies

In [ ]:
# reindex the years
for company in companies:
    for statement, data in company.items():
        if not statement == "df_ratio":
            company[statement] = company[statement].reindex(['2017', '2018', '2019', '2020', '2021', '2022', 'TTM'], axis=1)
        else:
            company[statement] = company[statement].reindex(['2017', '2018', '2019', '2020', '2021', '2022', 'Current'], axis=1)


### Convert unit to number

In [ ]:
unit_converter = {
    "K": 10 ** 3,
    "M": 10 ** 6,
    "B": 10 ** 9
}

def convert_to_number(data: str):
    try:
        number = data[:-1].replace("−", "-")
        if (unit := data[-1]) in unit_converter:
            return float(number) * unit_converter[unit]
        else:
            return float(data.replace("−", "-"))
    except:
        return np.nan

In [ ]:
genetec['df_inc']

In [ ]:
for statement, df in genetec.items(): 
    for col in df.columns:
        df[col] = df[col].apply(lambda x: convert_to_number(x))

In [ ]:
for statement, df in unisem.items():
    for col in df.columns:
        df[col] = df[col].apply(lambda x: convert_to_number(x))

In [ ]:
unisem['df_inc']

### Industry Average Comparison

<h3><u>Ratio Needed</u></h3>
Price to earnings ratio<br/>
Price to cash flow ratio<br/>
Price to book ratio<br/>
Return on assets %<br/>
Return on equity %<br/>
Return on invested capital %<br/>
Gross margin %<br/>
Operating margin %<br/>
Net margin %<br/>
Quick ratio<br/>
Current ratio<br/>
Inventory turnover<br/>
Asset turnover<br/>
Debt to assets ratio<br/>
Debt to equity ratio<br/>
Long term debt to total assets ratio<br/>
Long term debt to total equity ratio<br/>

#### Ratio Available on TradingView

In [ ]:
idx = pd.IndexSlice
# for value in set(unisem['df_ratio'].index.get_level_values(1)):  # check ratios available
#     print(value)
ratios = {}
for company, name in zip(companies, links):  # find the industry median of each ratio over 5 years
    ratio = company['df_ratio'].loc[idx[:, [
        "Price to earnings ratio",
        "Price to cash flow ratio",
        "Price to book ratio",
        "Return on assets %",
        "Return on equity %",
        "Return on invested capital %",
        "Gross margin %",
        "Operating margin %",
        "Net margin %",
        "Quick ratio",
        "Current ratio",
        "Inventory turnover",
        "Asset turnover",
        "Debt to assets ratio",
        "Debt to equity ratio",
        "Long term debt to total assets ratio",
        "Long term debt to total equity ratio",
    ]], :].unstack().median().to_frame().loc[idx["2018":, :], :]
    ratio.columns = ["Ratio"]
    ratios[name] = ratio
ratios

In [ ]:
for company, ratio in ratios.items(): # export to excel file
    filename = r"indsutry_average.xlsx"
    if not os.path.exists(filename):
        with pd.ExcelWriter(filename, mode='w', engine='openpyxl', if_sheet_exists="replace") as wf:
            ratio.to_excel(wf, sheet_name=company)
    else:
        with pd.ExcelWriter(filename, mode='a', engine='openpyxl', if_sheet_exists="replace") as wf:
            ratio.to_excel(wf, sheet_name=company)

#### PEG Ratio

In [ ]:
set(unisem['df_inc'].index.get_level_values(1)) # check what indicators on infcome statement is available

In [ ]:
unisem['df_inc'].loc[idx[:, "Diluted earnings per share (Diluted EPS)"], :]

In [ ]:
eps_growths = []
for company in companies:
    eps_growth = company['df_inc'].loc[idx[:, "Diluted earnings per share (Diluted EPS)"], :].pct_change(axis=1) * 100
    eps_growths.append(eps_growth)

In [ ]:
unisem['df_ratio'].loc[idx[:, "Price to earnings ratio"], :]

##### Unisem PEG Ratio Industry Average

In [ ]:
unisem['df_ratio'].loc[idx[:, "Price to earnings ratio"], :].unstack().rename(columns={"Current": "TTM"}).div(eps_growths[-1], axis=0, level=0)

##### Genetec PEG Ratio Industry Average

In [ ]:
def median(x):
    return np.nanmedian(x.loc[(np.isfinite(x))])

In [ ]:
genetec_peg_ia = genetec['df_ratio'].loc[idx[:, "Price to earnings ratio"], :].unstack().rename(columns={"Current": "TTM"}).div(eps_growths[0], axis=0, level=0).apply(lambda x: median(x)).unstack().rename(columns={"Price to earnings ratio": "Industry Average's PEG Ratio"})
genetec_peg_ia.to_clipboard(excel=True)
genetec_peg_ia

##### Unisem PEG Ratio Industry Average

In [ ]:
unisem_peg_ia = unisem['df_ratio'].loc[idx[:, "Price to earnings ratio"], :].unstack().rename(columns={"Current": "TTM"}).div(eps_growths[-1], axis=0, level=0).apply(lambda x: median(x), axis=0).unstack().rename(columns={"Price to earnings ratio": "Industry Average's PEG Ratio"})
unisem_peg_ia.to_clipboard(excel=True)
unisem_peg_ia

#### Collection Period

indicators needed
1. Total receivables, net
2. Total revenue

In [ ]:
# unisem['df_inc'].loc[idx[:, "Total revenue"], :].unstack().mul((unisem['df_bal'].loc[idx[:, "Total receivables, net"], :] / 365).unstack().values, axis=0)
genetec_acc = genetec['df_bal'].loc[idx[:, "Total receivables, net"], :].unstack()
genetec_sales = genetec['df_inc'].loc[idx[:, "Total revenue"], :].values
genetec_collection_period = ((genetec_acc / genetec_sales) * 365).median().unstack().rename(columns={"Total receivables, net": "Average Collection Period"})
genetec_collection_period.to_clipboard(excel=True)
genetec_collection_period

In [ ]:
# unisem['df_inc'].loc[idx[:, "Total revenue"], :].unstack().mul((unisem['df_bal'].loc[idx[:, "Total receivables, net"], :] / 365).unstack().values, axis=0)
unisem_acc = unisem['df_bal'].loc[idx[:, "Total receivables, net"], :].unstack()
unisem_sales = unisem['df_inc'].loc[idx[:, "Total revenue"], :].values
unisem_collection_period = ((unisem_acc / unisem_sales) * 365).median().unstack().rename(columns={"Total receivables, net": "Average Collection Period"})
unisem_collection_period.to_clipboard(excel=True)
unisem_collection_period

#### Days to Sells Inventory

Cost of goods sold
Total inventory

In [ ]:
idx = pd.IndexSlice
genetec_inv_copy = genetec['df_bal'].loc[idx[:, "Total inventory"], :].copy().drop("TTM", axis=1)
genetec_avg_inv = []
columns = genetec_inv_copy.columns
for i in range(len(columns) - 1):
    average_inv = list((genetec_inv_copy.loc[:, idx[columns[i] : columns[i+1]]].sum(axis=1) / 2 ).values)
    genetec_avg_inv.append(average_inv)
genetec_dio =  (1 / ((genetec['df_inc'].loc[idx[:, "Cost of goods sold"], :].drop(["2017", "TTM"], axis=1)).unstack(0).stack(0) / average_inv) * 365).unstack(0).unstack().unstack(2).median() * -1
genetec_dio.to_clipboard(excel=True)

In [ ]:
unisem_inv_copy = unisem['df_bal'].loc[idx[:, "Total inventory"], :].copy().drop("TTM", axis=1)
unisem_avg_inv = []
columns = unisem_inv_copy.columns
for i in range(len(columns) - 1):
    average_inv = list((unisem_inv_copy.loc[:, idx[columns[i] : columns[i+1]]].sum(axis=1) / 2 ).values)
    unisem_avg_inv.append(average_inv)
unisem_dio = ((1 / ((unisem['df_inc'].loc[idx[:, "Cost of goods sold"], :].drop(["2017", "TTM"], axis=1)).unstack(0).stack(0) / average_inv) * 365).unstack(0).unstack().unstack(2).median() * -1)
unisem_dio.to_clipboard(excel=True)

#### Cash Turnover

In [ ]:
genetec_revenue_copy = genetec['df_inc'].loc[idx[:, "Total revenue"], :].copy()
genetec_cash_copy = genetec["df_bal"].loc[idx[:, "Cash and short term investments"], :].drop("TTM", axis=1).copy()
genetec_avg_cash = []
columns = genetec_cash_copy.columns
for i in range(len(columns) - 1):
    average_cash = list((genetec_cash_copy.loc[:, idx[columns[i] : columns[i+1]]].sum(axis=1) / 2 ).values)
    genetec_avg_cash.append(average_cash)
(((genetec_revenue_copy.drop(["2017", "TTM"], axis=1)).unstack(0).stack(0) / np.array(genetec_avg_cash)).median(axis=1)).to_clipboard(excel=True)


In [ ]:
unisem_revenue_copy = unisem['df_inc'].loc[idx[:, "Total revenue"], :].copy()
unisem_cash_copy = unisem["df_bal"].loc[idx[:, "Cash and short term investments"], :].drop("TTM", axis=1).copy()
unisem_avg_cash = []
columns = unisem_cash_copy.columns
for i in range(len(columns) - 1):
    average_cash = list((unisem_cash_copy.loc[:, idx[columns[i] : columns[i+1]]].sum(axis=1) / 2 ).values)
    unisem_avg_cash.append(average_cash)
# (unisem_revenue_copy.drop(["2017", "TTM"], axis=1)).unstack(0).stack(0) / unisem_avg_cash
(((unisem_revenue_copy.drop(["2017", "TTM"], axis=1)).unstack(0).stack(0) / np.array(unisem_avg_cash)).median(axis=1)).to_clipboard(excel=True)


#### PPE Turnover

In [ ]:
genetec_cogs_copy = genetec['df_inc'].loc[idx[:, "Cost of goods sold"], :].copy() * -1
genetec_ppe_copy = genetec["df_bal"].loc[idx[:, "Net property/plant/equipment"], :].drop("TTM", axis=1).copy()
genetec_avg_ppe = []
columns = genetec_ppe_copy.columns
for i in range(len(columns) - 1):
    average_ppe = list((genetec_ppe_copy.loc[:, idx[columns[i] : columns[i+1]]].sum(axis=1) / 2 ).values)
    genetec_avg_ppe.append(average_ppe)
(((genetec_cogs_copy.drop(["2017", "TTM"], axis=1)).unstack(0).stack(0) / np.array(genetec_avg_ppe)).median(axis=1)).to_clipboard(excel=True)

In [ ]:
unisem_cogs_copy = unisem['df_inc'].loc[idx[:, "Cost of goods sold"], :].copy() * -1
unisem_ppe_copy = unisem["df_bal"].loc[idx[:, "Net property/plant/equipment"], :].drop("TTM", axis=1).copy()
unisem_avg_ppe = []
columns = genetec_ppe_copy.columns
for i in range(len(columns) - 1):
    average_ppe = list((unisem_ppe_copy.loc[:, idx[columns[i] : columns[i+1]]].sum(axis=1) / 2 ).values)
    unisem_avg_ppe.append(average_ppe)
(((unisem_cogs_copy.drop(["2017", "TTM"], axis=1)).unstack(0).stack(0) / np.array(unisem_avg_ppe)).median(axis=1)).to_clipboard(excel=True)

#### Working Capital Turnover

Total current assets <br/>
Total current liabilities

In [ ]:
genetec_net_working_capital = genetec['df_bal'].loc[idx[:, ["Total current assets", "Total current liabilities"]], :].groupby("Company", axis=0).sum().drop("TTM", axis=1)
genetec_avg_wc = []
columns = genetec_net_working_capital.columns
for i in range(len(columns) - 1):
    average_wc = list((genetec_net_working_capital.loc[:, columns[i]: columns[i+1]].sum(axis=1) / 2 ).values)
    genetec_avg_wc.append(average_wc)
(genetec['df_inc'].loc[idx[:, 'Total revenue'], :].drop(["2017", "TTM"], axis=1).T / np.array(genetec_avg_wc)).median(axis=1)
# genetec['df_inc'].loc[idx[:, 'Total revenue'], :].drop(["2017", "TTM"], axis=1).T
# genetec_avg_wc
(genetec['df_inc'].loc[idx[:, 'Total revenue'], :].drop(["2017", "TTM"], axis=1).T / np.array(genetec_avg_wc)).median(axis=1).to_clipboard(excel=True)

In [ ]:
unisem_net_working_capital = unisem['df_bal'].loc[idx[:, ["Total current assets", "Total current liabilities"]], :].groupby("Company", axis=0).sum().drop("TTM", axis=1)
unisem_avg_wc = []
columns = unisem_net_working_capital.columns
for i in range(len(columns) - 1):
    average_wc = list((unisem_net_working_capital.loc[:, columns[i]: columns[i+1]].sum(axis=1) / 2 ).values)
    unisem_avg_wc.append(average_wc)
((unisem['df_inc'].loc[idx[:, 'Total revenue'], :].drop(["2017", "TTM"], axis=1).T / np.array(unisem_avg_wc)).median(axis=1)).to_clipboard(excel=True)

#### Days Payable Outstanding

In [ ]:
for i in genetec['df_bal'].index.get_level_values(1):
    print(i)

In [ ]:
genetec_ap = genetec['df_bal'].loc[idx[:, "Accounts payable"], :].unstack()
genetec_sales = genetec['df_inc'].loc[idx[:, "Total revenue"], :].values
genetec_payable_period = ((genetec_ap / genetec_sales) * 365).median().unstack().rename(columns={"Accounts payable": "Average payble period"})
genetec_payable_period.to_clipboard(excel=True)
genetec_payable_period

In [ ]:
unisem_ap = unisem['df_bal'].loc[idx[:, "Accounts payable"], :].unstack()
unisem_sales = unisem['df_inc'].loc[idx[:, "Total revenue"], :].values
unisem_payable_period = ((unisem_ap / unisem_sales) * 365).median().unstack().rename(columns={"Accounts payable": "Average payble period"})
unisem_payable_period.to_clipboard(excel=True)
unisem_payable_period

#### Cash Conversion Cycle

In [ ]:
genetec_dio_f = genetec_dio.to_frame()
genetec_dio_f.name = "Indicator"
genetec_dio_f.columns = ["Days"]
unisem_dio_f = unisem_dio.to_frame()
unisem_dio_f.name = "Indicator"
unisem_dio_f.columns = ["Days"]
genetec_collection_period.columns = ['Days']
unisem_collection_period.columns = ['Days']
genetec_payable_period.columns = ['Days']
unisem_payable_period.columns = ['Days']

In [ ]:
(genetec_collection_period + genetec_dio_f - genetec_payable_period).to_clipboard(excel=True)
(genetec_collection_period + genetec_dio_f - genetec_payable_period)

In [ ]:
unisem_payable_period

In [ ]:
unisem_dio_f

In [ ]:
unisem_collection_period

In [ ]:
(unisem_collection_period + unisem_dio_f - unisem_payable_period).to_clipboard(excel=True)
(unisem_collection_period + unisem_dio_f - unisem_payable_period)

#### Times To Interest Earned

In [ ]:
unisem_ebit_interest = unisem['df_inc'].loc[idx[:, ["EBIT", "Interest expense, net of interest capitalized"]], :]
unisem_ebit_interest.groupby("Company", axis=0).agg(lambda x: x[0] / (-1 * x[-1])).median().to_frame().to_clipboard(excel=True)


In [ ]:
genetec_ebit_interest = genetec['df_inc'].loc[idx[:, ["EBIT", "Interest expense, net of interest capitalized"]], :]
genetec_ebit_interest.groupby("Company", axis=0).agg(lambda x: x[0] / (-1 * x[-1])).median().to_frame().to_clipboard(excel=True)